So choosing a frame work is key in bulding LLM Gen AI applications and also building AI agents.

Among those we a few which are `LlamaIndex` and `Langchain`.

with usecases like prompts, document loaders, vector stores.

Langchain operates on a high-level of abstraction

RAG - https://blogs.nvidia.com/blog/what-is-retrieval-augmented-generation/

In [10]:
import os

# os.environ["LANGCHAIN_TRACING_V2"]="true"
# os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
# os.environ["LANGCHAIN_PROJECT"]="Starting"
# os.environ["LANGCHAIN_API_KEY"]=os.environ.get("LANGSMITH_API_KEY")

In [11]:
from langsmith import traceable

In [12]:
from langchain_ollama import ChatOllama, OllamaLLM

In [13]:
model = ChatOllama(
    model = "llama3.2:latest"
)

In [14]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("You are a translator from English to Hindi. Translate the following from Englisht to Hindi. Just provide the translation."),
    HumanMessage("Hello, how are you?")
]

def run_llm():
    return model.invoke(messages)

print(run_llm())

content='नमस्ते! आप कैसे हैं?' additional_kwargs={} response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-01-31T16:38:40.514419Z', 'done': True, 'done_reason': 'stop', 'total_duration': 670992250, 'load_duration': 30926625, 'prompt_eval_count': 55, 'prompt_eval_duration': 250000000, 'eval_count': 14, 'eval_duration': 389000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-bf1d1572-f4fe-4cc2-b5b3-2fd6aaca55ff-0' usage_metadata={'input_tokens': 55, 'output_tokens': 14, 'total_tokens': 69}


#### For some reason langsmith is not working for me, weirdly enough...

In [15]:
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")])

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-01-31T16:38:41.346417Z', 'done': True, 'done_reason': 'stop', 'total_duration': 285115708, 'load_duration': 11689625, 'prompt_eval_count': 26, 'prompt_eval_duration': 27000000, 'eval_count': 10, 'eval_duration': 245000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-5054133e-e10b-4ff8-b7ee-9345273fccaf-0', usage_metadata={'input_tokens': 26, 'output_tokens': 10, 'total_tokens': 36})

In [16]:
for token in model.stream(messages):
    print(token.content, end="|")

न|म|स|्त|े,| आप| क|ैस|े| ह|ैं|?||

Documents and document loaders;

Text splitters;

Embeddings;

Vector stores and retrievers.

In [17]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [18]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../files/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


In [19]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'source': '../files/nke-10k-2023.pdf', 'page': 0}


In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

516

In [25]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3.2:latest")

In [50]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[50].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 3072

[0.026907003, -0.015096847, -0.021029899, -0.030414557, 0.010583798, -0.0260707, -0.018609315, -0.010385434, -0.002807863, -0.003978173]


In [51]:
print(vector_2)

[0.006000174, 0.023735197, 0.012992696, -0.032651838, 0.008283692, -0.013594953, 0.015199944, 0.0089687165, 0.023605661, -0.010202884, 0.003809493, 0.014063217, 0.010098865, 0.0023678076, -0.030351657, 0.0015021372, -0.007882115, -0.0081202155, 0.0023435801, 0.0033776686, -0.007581951, 0.0091411285, 0.033636555, -0.009991062, 0.003110617, 0.0030248573, -0.004691464, -0.024273504, -0.006887786, 0.030924316, -0.003355549, 0.021717783, -0.0005846555, -0.0017193214, -0.009467822, -0.011447344, -0.0007702978, -0.0022277345, 0.017939856, -0.01956104, 0.012871702, 0.036136884, 0.00736254, 0.0094643235, -0.018821845, -0.0043767793, -0.012643628, 0.0011547321, -0.0045949314, -0.0029309716, 0.013285754, 0.008184863, -0.011160422, 0.005991786, -0.0003855742, 0.011000478, 0.014087419, -0.029774733, 0.021079896, 0.011890698, -0.0063374047, 0.02972654, -0.012288709, -0.01797593, 0.01804677, -0.016562346, -0.006873507, -0.031553168, -0.00019523644, -0.025152858, -0.013691321, -0.012850657, -0.0134759

In [27]:
print(type(vector_1))

<class 'list'>


In [28]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.getenv("PINE_CONE_API_KEY"))

In [35]:
index_name = "quickstart"

pc.create_index(
    name=index_name,
    dimension=3072,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [36]:
from langchain_pinecone import PineconeVectorStore

index = pc.Index(index_name)

In [39]:
# Instantiated the PineconeVectorStore with the embeddings and the index

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [52]:
import numpy as np

valid_documents = []
valid_vectors = []

for doc in all_splits:
    vector = embeddings.embed_query(doc.page_content)
    if not np.all(np.array(vector) == 0.0):
        valid_documents.append(doc)
        valid_vectors.append(vector)

# if valid_documents:
#     ids = vector_store.add_documents(documents=valid_documents)
# else:
#     print("No valid documents to add.")

In [53]:
print(len(valid_documents))

516


In [57]:
for v in valid_vectors:
    if 0.00978250615 in v:
        print(v)

In [58]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='and growing our brands will depend on our design and marketing efforts, including advertising and consumer campaigns, product innovation and product quality. Our
commitment to product innovation, quality and sustainability, and our continuing investment in design (including materials), marketing and sustainability measures may not
have the desired impact on our brand image and reputation. In addition, our success in maintaining, extending and expanding our brand image depends on our ability to
adapt to a rapidly changing media and digital environment, including our reliance on social media and other digital advertising networks, and digital dissemination of
advertising campaigns on our digital platforms and through our digital experiences and products. We could be adversely impacted if we fail to achieve any of these
objectives.' metadata={'page': 15.0, 'source': '../files/nke-10k-2023.pdf', 'start_index': 3705.0}


In [59]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='Notes to Consolidated Financial Statements 60
2. Financial Statement Schedule:
II — Valuation and Qualifying Accounts for the years ended May 31, 2023, 2022 and 2021 96
All other schedules are omitted because they are not applicable or the required information is shown in the financial statements ornotes thereto.
3. Exhibits:
3.1 Restated Articles of Incorporation, as amended (incorporated by reference to Exhibit 3.1 to the Company's Quarterly Report on Form 10-Q for the fiscalquarter ended November 30, 2015).
3.2 Fifth Restated Bylaws, as amended (incorporated by reference to Exhibit 3.1 to the Company's Current Report on Form 8-K filed June 19, 2020).
4.1 Restated Articles of Incorporation, as amended (see Exhibit 3.1).
4.2 Fifth Restated Bylaws, as amended (see Exhibit 3.2).
4.3 Indenture dated as of April 26, 2013, by and between NIKE, Inc. and Deutsche Bank Trust Company Americas, as trustee (incorporated by reference toExhibit 4.1 to the Company's Form 8-K filed Apr

In [60]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.375297904

page_content='Global Brand Divisions' loss before interest and taxes increased 14% for fiscal 2023 primarily due to a 12% increase in selling and administrative expense from higher
operating overhead expense largely driven by higher wage-related costs and strategic technology enterprise investments.
CONVERSE
(Dollars in millions) FISCAL 2023 FISCAL 2022 % CHANGE
% CHANGEEXCLUDINGCURRENCYCHANGESFISCAL 2021 % CHANGE
% CHANGEEXCLUDINGCURRENCYCHANGES
Revenues by:
Footwear $ 2,155 $ 2,094 3 % 8 %$ 1,986 5 % 6 %
Apparel 90 103 -13 % -7 % 104 -1 % -3 %
Equipment 28 26 8 % 16 % 29 -10 % -16 %
Other 154 123 25 % 25 % 86 43 % 42 %
TOTAL REVENUES $ 2,427 $ 2,346 3 % 8 %$ 2,205 6 % 7 %
Revenues by:
Sales to Wholesale Customers $ 1,299 $ 1,292 1 % 7 %$ 1,353 -5 % -4 %
Sales through Direct to Consumer 974 931 5 % 8 % 766 22 % 22 %
Other 154 123 25 % 25 % 86 43 % 42 %
TOTAL REVENUES $ 2,427 $ 2,346 3 % 8 %$ 2,205 6 % 7 %
EARNINGS BEFORE INTEREST AND TAXES $ 676 $ 669 1 % $ 543 23 

In [63]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.asimilarity_search_by_vector(embedding)
print(results)

<coroutine object VectorStore.asimilarity_search_by_vector at 0x1103dbc40>


/var/folders/dt/nf7j33w51gj7cv6bx6633tm40000gn/T/ipykernel_66738/388494021.py:3: RuntimeWarning: coroutine 'VectorStore.asimilarity_search_by_vector' was never awaited
  results = vector_store.asimilarity_search_by_vector(embedding)


In [64]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='4682d224-0cca-4d11-9a0a-a43876310491', metadata={'page': 15.0, 'source': '../files/nke-10k-2023.pdf', 'start_index': 3705.0}, page_content='and growing our brands will depend on our design and marketing efforts, including advertising and consumer campaigns, product innovation and product quality. Our\ncommitment to product innovation, quality and sustainability, and our continuing investment in design (including materials), marketing and sustainability measures may not\nhave the desired impact on our brand image and reputation. In addition, our success in maintaining, extending and expanding our brand image depends on our ability to\nadapt to a rapidly changing media and digital environment, including our reliance on social media and other digital advertising networks, and digital dissemination of\nadvertising campaigns on our digital platforms and through our digital experiences and products. We could be adversely impacted if we fail to achieve any of these\nobjectives.

In [65]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(id='4682d224-0cca-4d11-9a0a-a43876310491', metadata={'page': 15.0, 'source': '../files/nke-10k-2023.pdf', 'start_index': 3705.0}, page_content='and growing our brands will depend on our design and marketing efforts, including advertising and consumer campaigns, product innovation and product quality. Our\ncommitment to product innovation, quality and sustainability, and our continuing investment in design (including materials), marketing and sustainability measures may not\nhave the desired impact on our brand image and reputation. In addition, our success in maintaining, extending and expanding our brand image depends on our ability to\nadapt to a rapidly changing media and digital environment, including our reliance on social media and other digital advertising networks, and digital dissemination of\nadvertising campaigns on our digital platforms and through our digital experiences and products. We could be adversely impacted if we fail to achieve any of these\nobjectives.

In [66]:
# Noted issues are PineCone should not have docs wiht hugely sparse vectors